In [1]:
import tensorflow as tf
import numpy as np
import mynn as mynn


In [2]:
archetypes = []
header = None
cards = None
classes = set()
import csv
with open('archetypes.csv', 'rb') as csvfile:
  reader = csv.reader(csvfile, delimiter=',', quotechar='"')
  for row in reader:
    if header == None:
      header = row
      cards = map(int, row[3:])
    elif row[2] == "23":
      c = int(row[1])
      classes.add(c)
      rec = [map(float, row[3:]), c]
      archetypes.append(rec)
    else:
      print row[2];
      #break

In [3]:
io_pair = []
max_samples = 100
import random
random.shuffle(archetypes)
for rec in archetypes[:max_samples]:
    io_pair.append([
        rec[0],
        map(lambda x: (1.0 if x == rec[1] else 0.0), classes)
    ])
len(io_pair)

100

In [4]:
split = int(len(io_pair) * 0.8)
train = io_pair[:split]
test  = io_pair[(split+1):]


In [5]:
print "Training size: ", len(train)
print "Testing size:  ", len(test)

Training size:  80
Testing size:   19


In [6]:
def derivsig(x):
    s = tf.nn.sigmoid(x)
    return tf.mul(s, tf.sub(tf.ones(tf.shape(x)), s))
net = mynn.MyNeuralNet(tf.nn.softmax, tf.nn.sigmoid, derivsig )
net.init_weights(len(cards), 20, None, len(classes))
net.set_classes(list(classes))

In [7]:
import time
for sample in train:   
  e = net.train(sample[0], sample[1])

start_time = time.time()
net.finish_training()
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.65481090546 seconds ---


In [9]:
errors = 0.0
start_time = time.time()
batch_size = 100
placeholder = tf.placeholder(tf.float32, shape=(batch_size, net.input_size))
eval_op = net.evaluate_op(placeholder)
best_op = tf.argmax(tf.transpose(placeholder), 0)
with tf.Session() as sess:
  
  res  = sess.run(eval_op, feed_dict={
    placeholder: map(lambda x: x[0], test),
  })
  best = sess.run(best_op, feed_dict={
    placeholder: res,
  })
  correct_count = tf.reduce_sum(tf.cast(tf.nn.in_top_k(best, map(lambda x: x[1], test), 1), tf.int32))
  print correct_count
print "Error rate ", errors/len(test)
print("--> %s seconds ---" % (time.time() - start_time))
#net.close()

ValueError: Shapes (100,) and (1,) are not compatible